## Set to directory with python scripts

In [1]:
scripts_path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts/'

# Path to save figures
path_to_save = '/'.join(scripts_path.split('/')[:-2]) + '/'

# Path to save supplementary and figure data folders
save_path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/'
%cd $scripts_path

/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts


## Load modules

In [2]:
import scipy.stats as st
from collections import Counter
import sklearn
from matplotlib import cm
import scipy
import  itertools
from datetime import datetime
from seaborn import clustermap
from scipy.cluster.hierarchy import linkage
from sklearn.linear_model import LogisticRegression
import os
import time
import pickle as pkl
import pandas as pd
from dataLoader import *
from basic_data_methods_helper import *
from skbio.stats.distance import permanova, DistanceMatrix, anosim
from skbio.stats.distance import mantel
from statsmodels.stats.multitest import multipletests
from matplotlib.collections import LineCollection
from Bio import Phylo
import re
from statistics import mode
import seaborn as sns
from sksurv.linear_model import CoxnetSurvivalAnalysis, CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
import matplotlib.lines as mlines
import seaborn as sns
from skbio.diversity.alpha import chao1
import statsmodels.api as sm

# Set font for figures
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})

## Load data

In [3]:
# Input your own user path here
user_path = "/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/"

In [4]:
path = user_path + 'scripts/inputs'
filename_demo = 'pt_demographics_full.xlsx'
df_demo = pd.read_excel(path + '/' + filename_demo, sheet_name = 'Sheet1', index_col = 0)
for col in df_demo.columns.values:
    df_demo = df_demo.rename(columns={col: col.split('  ')[0]})
    df_demo = df_demo.rename(columns={col: col.split(' (')[0]})

In [5]:
# set data_path to point to directory with data
data_path = save_path + "Data/"

# Option to change filtering criteria
dl = dataLoader(path = data_path, pt_perc = {'metabs': .25, '16s': .1, 'scfa': 0, 'toxin':0}, meas_thresh = 
                {'metabs': 0, '16s': 10, 'scfa': 0, 'toxin':0}, 
                var_perc = {'metabs': 50, '16s': 5, 'scfa': 0, 'toxin':0}, pt_tmpts = 1)

No patient demographic/clinical data available due to identifiability


In [6]:
# Number of features in each data set
for key in dl.week.keys():
    print(key + ': ' + str(dl.week[key][0]['x'].shape[1]))

metabs: 760
16s: 237
scfa: 8
toxin: 2
metabs_16s: 997
metabs_16s_scfa: 997
metabs_toxin: 762


# Table 1

### Table 1 - Demographics

In [ ]:
# NOTE: This analysis will not be run-able with the data provided, as the demographic data was not included due to possible issues with identifiability

In [15]:
targets = pd.Series(dl.targets_by_pt)
targets = targets.loc[dl.demographics.index.values]
targets = targets.replace('Cleared','Non-recurrer')
targets = targets.replace('Recur', 'Recurrer')
demo_dict = {}
for feat in dl.demographics.columns.values:
    if 'categories' in feat:
        continue
    if len(np.unique(dl.demographics[feat]))<=4 or (dl.demographics[feat].dtypes!=int and dl.demographics[feat].dtypes!=float):
        for cat in np.unique(dl.demographics[feat]):
            N = np.sum((dl.demographics[feat]==cat)*(targets=='Non-recurrer'))
            c_re = np.sum((dl.demographics[feat]==cat)*(targets=='Recurrer'))
            demo_dict[(feat,cat)] = {'Non-recurrer': N, 'Recurrer': c_re}
    else:
        for outcome in ['Recurrer', 'Non-recurrer']:
            ix = targets == outcome
            if (feat, 'Mean') not in demo_dict.keys():
                demo_dict[(feat,'Mean')] = {}
                demo_dict[(feat,'STD')] = {}
                demo_dict[(feat,'Median')] = {}
                demo_dict[(feat,'Range')] = {}
            demo_dict[(feat,'Mean')][outcome]= np.mean(dl.demographics[feat][ix])
            demo_dict[(feat,'STD')][outcome] =  np.std(dl.demographics[feat][ix])
            demo_dict[(feat,'Median')][outcome] = np.median(dl.demographics[feat][ix])
            demo_dict[(feat,'Range')][outcome]= (np.min(dl.demographics[feat][ix]),np.max(dl.demographics[feat][ix]))
        
df = pd.DataFrame(demo_dict).T
if not os.path.isdir(save_path + 'Notes'):
    os.mkdir(save_path + 'Supplemental Tables')
df.to_csv(save_path + 'Notes/Demographics.csv')

/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expre

/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expre

/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expre

### Table 1 - # Recurrers at each timepoint

In [39]:
rc_dict = {}
for week in dl.week['metabs'].keys():
    rc_dict[week]={}
    targets = dl.week['metabs'][week]['y']
    rc_dict[week]['Recurrers'] = np.sum(targets=='Recurrer')
    rc_dict[week]['Non-Recurrers'] = np.sum(targets=='Non-recurrer')
if not os.path.isdir(path_to_save + 'output_tables'):
    os.mkdir(path_to_save + 'output_tables')
pd.DataFrame(rc_dict).T.to_csv(save_path + 'Main Tables/Table 2.csv')

# Univariate analysis

## Univariate analysis for demographics and clinical variables

In [128]:
if not os.path.isdir(path_to_save + 'univariate_analysis'):
    os.mkdir(path_to_save + 'univariate_analysis')

# Univariate analysis for demographics
targets = dl.keys['metabs']['targets_by_pt'].loc[dl.demographics.index.values]
x = dl.demographics
xx = dl.demographics[['Age','Sex','Race','BMI','Smoking status', 'Prior PPI use','Drug','History of IBS?',
                     'Previous CDI dx?','Hx of liver disease', 'Hx of kidney disease', 
                      'Diabetes\n(1=pre-diabetes, 2=diabetes)','HX of CVD','Toxin +',
                     'Asthma','Obesity','Hypercholesterolemia','Anemia','GERD','Depression, anxiety, and/or PTSD',
                     'Malignacies','Arthritis','Hyperlipidemia','Hypersensitivity lung disease', 'Diverticulitis']]
df_out = univariate_w_chi2(xx, targets, chi_sqr = True)
df_out.to_csv(path_to_save + 'univariate_analysis/demographics.csv')

AttributeError: 'dataLoader' object has no attribute 'demographics'

In [12]:
# Univariate analysis for clinical variables
targets = dl.keys['metabs']['targets_by_pt'].loc[dl.clinical.index.values]
x = dl.clinical
df_out = univariate_w_chi2(x, targets, chi_sqr = True)
df_out.to_csv(path_to_save + 'univariate_analysis/clinical.csv')

## Export 16S data for DESeq2 in R

In [129]:
path_to_save_data=save_path + '/GenFigure&AnalysesData/'

if not os.path.isdir(path_to_save + 'scripts/inputs/'):
    os.mkdir(path_to_save + 'scripts/inputs/')
    
if not os.path.isdir(path_to_save + 'scripts/inputs/DEseq2_covariate'):
    os.mkdir(path_to_save + 'scripts/inputs/DEseq2_covariate')

weeks = [0,1,2]
plot = True

data_to_save = {}
key = '16s'
if key not in data_to_save.keys():
    data_to_save[key] = {}
for week in weeks:
#     if not isinstance(week, list):
    x = dl.week_filt[key][week]['x']
    y = (dl.week_filt[key][week]['y']=='Recurrer').astype('float')
    y2 = (df_demo['Drug']=='vanc').astype('float')
    y2.index = [str(ix) + '-' + str(week) for ix in y2.index.values]
    y2 = y2.loc[x.index.values]
    y.index = [str(ix) + '-' + str(week) for ix in y.index.values]
    y2.name = 'Drug\n(Vanc=1, Metro=0)'
#     else:
#         x,y,t = get_slope_data(dl.week_filt[key], week)
        
    if not os.path.isdir('inputs/DEseq2/'):
        os.mkdir('inputs/DEseq2/')
    x.to_csv(path_to_save +'scripts/inputs/DEseq2_covariate/counts_' + str(week).replace('.','-') + '.csv')
    y.to_csv(path_to_save +'scripts/inputs/DEseq2_covariate/targ_' + str(week).replace('.','-') + '.csv')
    y2.to_csv(path_to_save + 'scripts/inputs/DEseq2_covariate/drug_' + str(week).replace('.','_') + '.csv')
    yy = pd.DataFrame(y, columns = ['Outcome\n(R=1,NR=0)'])
    if week != 0:
        both = pd.concat([x, yy, y2], 1)
    else:
        both = pd.concat([x, yy], 1)
    if 'Week ' + str(week) not in data_to_save[key].keys():
        data_to_save[key]['Week ' + str(week)] = {}
    data_to_save[key]['Week ' + str(week)]['RvNR'] = both
    
weeks = [(0,1),(1,2)]
key = '16s'
for outcome in ['Recurrer', 'Non-recurrer']:
    for week_pair in weeks:
        col0 = dl.week_filt[key][week_pair[0]]['x'].columns.values
        col1 = dl.week_filt[key][week_pair[1]]['x'].columns.values
        col_all = np.unique(np.concatenate([col0, col1]))
        ix0, ix1 = dl.week_filt[key][week_pair[0]]['x'].index.values, dl.week_filt[key][week_pair[1]]['x'].index.values
        
        x0, y0 = dl.keys[key]['data'][col_all].loc[ix0], dl.week_filt[key][week_pair[0]]['y']
        x1, y1 = dl.keys[key]['data'][col_all].loc[ix1], dl.week_filt[key][week_pair[1]]['y']
        x0 = x0.loc[y0.values == outcome]
        x1 = x1.loc[y1.values == outcome]
        x = pd.concat([x0, x1])
        y = [week_pair[0]]*x0.shape[0] + [week_pair[1]]*x1.shape[0]
        y = pd.Series((np.array(y)==week_pair[1]).astype('float'), index = x.index.values)
        
        y2 = (df_demo['Drug']=='vanc').astype('float')
        y2.index = [str(ix) for ix in y2.index.values]
        x_reps = [ix.split('-')[0] for ix in x.index.values]
        y2 = y2[x_reps]
        y2.index = x.index.values
        y2.name = 'Drug\n(Vanc=1, Metro=0)'
#         y2 = y2.loc[x.index.values]
        if not os.path.isdir('inputs/DEseq2/'):
            os.mkdir('inputs/DEseq2/')
        x.to_csv(path_to_save +'scripts/inputs/DEseq2_covariate/counts_' + outcome +str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')
        y.to_csv(path_to_save +'scripts/inputs/DEseq2_covariate/targ_' + outcome + str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')
        y2.to_csv(path_to_save +'scripts/inputs/DEseq2_covariate/drug_' + outcome + str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')

        yy = pd.DataFrame(y, columns = ['Outcome\n(R=1, NR=0)'])
        both = pd.concat([x,yy],1)
        if outcome not in data_to_save[key].keys():
            data_to_save[key][outcome] = {}
        data_to_save[key][outcome]['Week ' + str(week_pair[0]) + 'v' + str(week_pair[1])] = both
        
for key in data_to_save.keys():
    for key2 in data_to_save[key].keys():
        for key3 in data_to_save[key][key2].keys():
            val = pd.DataFrame(data_to_save[key][key2][key3])
            if os.path.isfile(path_to_save_data + '/' + 'univ_analysis_wcontrol.xlsx'):
                with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_wcontrol.xlsx', mode='a', engine='openpyxl') as writer:
                    val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
            else:
                with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_wcontrol.xlsx', mode = 'w', engine='openpyxl') as writer:
                    val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))

In [20]:
# path_to_save_data=save_path + '/GenFigure&AnalysesData/'

# if not os.path.isdir(path_to_save + 'scripts/inputs/'):
#     os.mkdir(path_to_save + 'scripts/inputs/')
    
# if not os.path.isdir(path_to_save + 'scripts/inputs/DEseq2'):
#     os.mkdir(path_to_save + 'scripts/inputs/DEseq2')

# weeks = [0,1,2]
# plot = True

# data_to_save = {}
# key = '16s'
# if key not in data_to_save.keys():
#     data_to_save[key] = {}
# for week in weeks:
# #     if not isinstance(week, list):
#     x = dl.week_filt[key][week]['x']
#     y = (dl.week_filt[key][week]['y']=='Recurrer').astype('float')
# #     else:
# #         x,y,t = get_slope_data(dl.week_filt[key], week)
        
#     if not os.path.isdir('inputs/DEseq2/'):
#         os.mkdir('inputs/DEseq2/')
#     x.to_csv(path_to_save +'scripts/inputs/DEseq2/counts_' + str(week).replace('.','-') + '.csv')
#     y.to_csv(path_to_save +'scripts/inputs/DEseq2/col_' + str(week).replace('.','-') + '.csv')
#     yy = pd.DataFrame(y, columns = ['Outcome'])
#     both = pd.concat([x, yy, yy2], 1)
#     if 'Week ' + str(week) not in data_to_save[key].keys():
#         data_to_save[key]['Week ' + str(week)] = {}
#     data_to_save[key]['Week ' + str(week)]['RvNR'] = both
    
# # Export for DEseq2 in R
# weeks = [(0,1),(1,2)]
# key = '16s'
# for outcome in ['Recurrer', 'Non-recurrer']:
#     for week_pair in weeks:
#         col0 = dl.week_filt[key][week_pair[0]]['x'].columns.values
#         col1 = dl.week_filt[key][week_pair[1]]['x'].columns.values
#         col_all = np.unique(np.concatenate([col0, col1]))
#         ix0, ix1 = dl.week_filt[key][week_pair[0]]['x'].index.values, dl.week_filt[key][week_pair[1]]['x'].index.values
        
#         x0, y0 = dl.keys[key]['data'][col_all].loc[ix0], dl.week_filt[key][week_pair[0]]['y']
#         x1, y1 = dl.keys[key]['data'][col_all].loc[ix1], dl.week_filt[key][week_pair[1]]['y']
#         x0 = x0.loc[y0.values == outcome]
#         x1 = x1.loc[y1.values == outcome]
#         x = pd.concat([x0, x1])
#         y = [week_pair[0]]*x0.shape[0] + [week_pair[1]]*x1.shape[0]
#         y = pd.Series((np.array(y)==week_pair[1]).astype('float'), index = x.index.values)
#         if not os.path.isdir('inputs/DEseq2/'):
#             os.mkdir('inputs/DEseq2/')
#         x.to_csv(path_to_save +'scripts/inputs/DEseq2/counts_' + outcome +str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')
#         y.to_csv(path_to_save +'scripts/inputs/DEseq2/col_' + outcome + str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')
        
#         yy = pd.DataFrame(y, columns = ['Outcome'])
#         both = pd.concat([x, yy], 1)
#         if outcome not in data_to_save[key].keys():
#             data_to_save[key][outcome] = {}
#         data_to_save[key][outcome]['Week ' + str(week_pair[0]) + 'v' + str(week_pair[1])] = both
        
# for key in data_to_save.keys():
#     for key2 in data_to_save[key].keys():
#         for key3 in data_to_save[key][key2].keys():
#             val = pd.DataFrame(data_to_save[key][key2][key3])
#             if os.path.isfile(path_to_save_data + '/' + 'univ_analysis_data.xlsx'):
#                 with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_data.xlsx', mode='a', engine='openpyxl') as writer:
#                     val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
#             else:
#                 with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_data.xlsx', mode = 'w', engine='openpyxl') as writer:
#                     val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
        

### Export 16S data for DESeq2 in R, with drug type as response variable

In [45]:
# path_to_save_data=save_path + '/GenFigure&AnalysesData/'

# if not os.path.isdir(path_to_save + 'scripts/inputs/'):
#     os.mkdir(path_to_save + 'scripts/inputs/')
    
# if not os.path.isdir(path_to_save + 'scripts/inputs/DEseq2'):
#     os.mkdir(path_to_save + 'scripts/inputs/DEseq2')

# weeks = [0,1,2]
# plot = True

# data_to_save = {}
# key = '16s'
# if key not in data_to_save.keys():
#     data_to_save[key] = {}
# for week in weeks:
# #     if not isinstance(week, list):
#     x = dl.week_filt[key][week]['x']
#     y = (df_demo['Drug']=='vanc').astype('float')
#     y.index = [str(ix) + '-' + str(week) for ix in y.index.values]
#     y = y.loc[x.index.values]
        
#     if not os.path.isdir('inputs/DEseq2_drug/'):
#         os.mkdir('inputs/DEseq2_drug/')
#     x.to_csv(path_to_save +'scripts/inputs/DEseq2_drug/counts_' + str(week).replace('.','-') + '.csv')
#     y.to_csv(path_to_save +'scripts/inputs/DEseq2_drug/col_' + str(week).replace('.','-') + '.csv')
#     yy = pd.DataFrame(y, columns = ['Outcome'])
#     both = pd.concat([x, yy], 1)
#     if 'Week ' + str(week) not in data_to_save[key].keys():
#         data_to_save[key]['Week ' + str(week)] = {}
#     data_to_save[key]['Week ' + str(week)]['VancVsMet'] = both
    
# # Export for DEseq2 in R
# weeks = [(0,1),(1,2)]
# key = '16s'
# for outcome in ['vanc', 'flagyl']:
#     for week_pair in weeks:
#         col0 = dl.week_filt[key][week_pair[0]]['x'].columns.values
#         col1 = dl.week_filt[key][week_pair[1]]['x'].columns.values
#         col_all = np.unique(np.concatenate([col0, col1]))
#         ix0, ix1 = dl.week_filt[key][week_pair[0]]['x'].index.values, dl.week_filt[key][week_pair[1]]['x'].index.values
        
#         x0 = dl.keys[key]['data'][col_all].loc[ix0]
#         y0 = df_demo['Drug'].copy()
#         y0.index = [str(ix) + '-' + str(week_pair[0]) for ix in y0.index.values]
#         y0 = y0.loc[x0.index.values]
        
#         x1 = dl.keys[key]['data'][col_all].loc[ix1]
#         y1 = df_demo['Drug'].copy()
#         y1.index = [str(ix) + '-' + str(week_pair[1]) for ix in y1.index.values]
#         y1 = y1.loc[x1.index.values]
        
#         x0 = x0.loc[y0.values == outcome]
#         x1 = x1.loc[y1.values == outcome]
#         x = pd.concat([x0, x1])
#         y = [week_pair[0]]*x0.shape[0] + [week_pair[1]]*x1.shape[0]
#         y = pd.Series((np.array(y)==week_pair[1]).astype('float'), index = x.index.values)
#         if not os.path.isdir('inputs/DEseq2_drug/'):
#             os.mkdir('inputs/DEseq2_drug/')
#         x.to_csv(path_to_save +'scripts/inputs/DEseq2_drug/counts_' + outcome +str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')
#         y.to_csv(path_to_save +'scripts/inputs/DEseq2_drug/col_' + outcome + str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')
        
#         yy = pd.DataFrame(y, columns = ['Outcome'])
#         both = pd.concat([x, yy], 1)
#         if outcome not in data_to_save[key].keys():
#             data_to_save[key][outcome] = {}
#         data_to_save[key][outcome]['Week ' + str(week_pair[0]) + 'v' + str(week_pair[1])] = both
        
# for key in data_to_save.keys():
#     for key2 in data_to_save[key].keys():
#         for key3 in data_to_save[key][key2].keys():
#             val = pd.DataFrame(data_to_save[key][key2][key3])
#             if os.path.isfile(path_to_save_data + '/' + 'univ_analysis_drug_response.xlsx'):
#                 with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_drug_response.xlsx', mode='a', engine='openpyxl') as writer:
#                     val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
#             else:
#                 with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_drug_response.xlsx', mode = 'w', engine='openpyxl') as writer:
#                     val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
        

## Reformat DEseq2 results

In [185]:
# Set path_deseq2 to path with R results
# path_deseq2 = path_to_save + 'univariate_analysis/16s/deseq2/'
path_deseq2 = path_to_save + 'univariate_analysis_control/16s/deseq2/'

weeks = [0,1,2]
plot = False

key = '16s'
    
if not os.path.isdir(path_to_save + 'univariate_analysis_control/' + key):
    os.mkdir(path_to_save + 'univariate_analysis_control/' + key)
for file in os.listdir(path_deseq2):
    if file == '.DS_Store':
        continue
    week = file.split('res')[-1].split('.')[0]
#     if os.path.isfile(path_to_save+'univariate_analysis_control/' + key + '/deseq2_' + key + str(week) + '.csv'):
#         continue
    df_r = pd.read_csv(path_deseq2 + file, index_col = 0)
    df_sorted = df_r.sort_values(by = 'padj')
    taxa = return_taxa_names(df_sorted.OTU)
    df_sorted.insert(1, "Taxa", taxa, allow_duplicates=True)
    
    df_sorted.insert(2,"Direction", np.zeros(df_sorted.shape[0]).astype(str), True)
    
    if '_' in file:
        # Higher in cleared group (0) or first week
        df_sorted["Direction"].loc[df_sorted["log2fold"]<0] = 'Higher in week ' + file.split('_')[0].split(
                'rer')[1]
#         except:
#             df_sorted["Direction"].loc[df_sorted["log2fold"]<0] = 'Higher in week ' + file.split('_')[0].split(
#                 'rer')[1]

        # Higher in recurred group (1) or second week
        df_sorted["Direction"].loc[df_sorted["log2fold"]>0] = 'Higher in week ' + file.split('_')[1].split('.')[0]
    else:
        # Higher in cleared group (0) or first week
        df_sorted["Direction"].loc[df_sorted["log2fold"]<0] = 'Decreased Risk'

        # Higher in recurred group (1) or second week
        df_sorted["Direction"].loc[df_sorted["log2fold"]>0] = 'Increased Risk'
    
    df_sorted = df_sorted.set_index("OTU")
    if plot:
        for metab in df_sorted.index.values:
            if df_sorted['padj'][metab] < .1:
                plot_metab_over_time(metab, df_sorted['padj'][metab], dl.keys[key], 
                                     path_to_save + 'univariate_analysis_control/' + 
                                     key + '/' + str(week) + '_deseq2_')
    df_sorted.to_csv(path_to_save+'univariate_analysis_control/' + key + '/deseq2_' + key + str(week) + '.csv')
    plt.figure()
    plt.hist(df_sorted['padj'], bins = np.arange(0,1.05,.05))
    plt.xlabel('p-values')
    plt.ylabel('Frequency')
    plt.title(key + ', week ' + str(week))
    plt.savefig(path_to_save+'univariate_analysis_control/' + key + '/deseq2_' + key + str(week) + '.pdf')
    plt.close()
    

/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

## Univariate analysis for metabolites & SCFAs

In [132]:
# Univariate analysis for metabolites and SCFAs
path_to_save_data=save_path + '/GenFigure&AnalysesData/'

weeks = [0,1,2]
week_pairs = [(0,1),(1,2)]
plot = False
data_to_save = {}
for key in ['metabs', 'scfa']:
    data_to_save[key] = {}
    if not os.path.isdir(path_to_save + 'univariate_analysis_control/' + key):
        os.mkdir(path_to_save + 'univariate_analysis_control/' + key)
    for outcome in ['Recurrer', 'Non-recurrer']:
        if outcome not in data_to_save[key].keys():
            data_to_save[key][outcome] = {}
        for week_pair in week_pairs:
            x0, y0 = dl.week[key][week_pair[0]]['x'], dl.week[key][week_pair[0]]['y']
            x0 = x0.loc[y0.values == outcome]
            x1, y1 = dl.week[key][week_pair[1]]['x'], dl.week[key][week_pair[1]]['y']
            x1 = x1.loc[y1.values == outcome]

            xboth = list(set(x0.columns.values).intersection(set(x1.columns.values)))
            x = pd.concat([x0, x1])
            if 'scfa' in key:
                x = x.drop('Caproate', axis = 1)
            y = [week_pair[0]]*x0.shape[0] + [week_pair[1]]*x1.shape[0]
            y = pd.Series((np.array(y)==week_pair[1]).astype('float'), index = x.index.values)
            
            y2 = (df_demo['Drug']=='vanc').astype('float')
            y2.index = [str(ix) for ix in y2.index.values]
            x_reps = [ix.split('-')[0] for ix in x.index.values]
            y2 = y2.loc[x_reps]
            y2.index = x.index.values
            y2.name = 'Drug\n(Vanc=1, Metro=0)'
            
            yy = pd.DataFrame(y, columns = ['Week'])
            both = pd.concat([x, yy, y2], 1)
            data_to_save[key][outcome]['Week ' + str(week_pair[0]) + 'v' + str(week_pair[1])] = both
            
            
            data_exog = pd.concat([yy, y2], 1)
            data_exog = sm.add_constant(data_exog)
            res_dict = {}
            for metab in x.columns.values:
                res_dict[metab] = {}
                model = sm.OLS(x[metab], data_exog)
                results = model.fit()
                cont_res = results.t_test([0,1,0])
                df = cont_res.summary_frame()
                res_dict[metab]['p, week'] = df['P>|t|']['c0']
                res_dict[metab]['coef, week'] = results.params['Week']
                df_2 = results.t_test([0,0,1]).summary_frame()
                res_dict[metab]['p, treatment'] = df_2['P>|t|']['c0']
                res_dict[metab]['coef, treatment'] = results.params['Drug\n(Vanc=1, Metro=0)']

            final_df = pd.DataFrame(res_dict).T
            reject, corrected, a1, a2 = multipletests(final_df['p, week'], alpha=.05, method='fdr_bh')
            reject2, corrected2, a12, a22 = multipletests(final_df['p, treatment'], alpha=.05, method='fdr_bh')
            final_df.insert(1, 'FDR, Week', corrected)
            final_df.insert(4, 'FDR, Treatment', corrected2)
            
            if not os.path.isdir(path_to_save+'univariate_analysis_control/' + key ):
                os.mkdir(path_to_save+'univariate_analysis_control/' + key)
#             with pd.ExcelWriter(path_save + 'Table 5 - Predictive Results.xlsx', mode = 'a') as writer:
#                 df.to_excel(writer, sheet_name = 'A. Results')
            final_df.to_csv(path_to_save+'univariate_analysis_control/' + key + '/OLS_' + key + '_' + outcome + '_' +
                            str(week_pair[0]) + 'v' + str(week_pair[1]) + '.csv')
        
    for week in weeks:
        if week not in data_to_save[key].keys():
            data_to_save[key]['Week ' + str(week)] = {}

        x, y = dl.week[key][week]['x'], dl.week[key][week]['y']

        yy = pd.Series((np.array(y)=='Recurrer').astype('float'), index = x.index.values)
        yy = pd.DataFrame(yy, columns = ['Outcome\n(R=1,NR=0)'])

        y2 = (df_demo['Drug']=='vanc').astype('float')
        y2.index = [str(ix) + '-' + str(week) for ix in y2.index.values]
        y2 = y2.loc[x.index.values]
        y2.name = 'Drug\n(Vanc=1, Metro=0)'
        # y2.index = x.index.values

        if week != 0:
            both = pd.concat([x, yy, y2], 1)
            data_exog = pd.concat([yy, y2], 1)
        else:
            both = pd.concat([x, yy], 1)
            data_exog = yy
        data_to_save[key]['Week ' + str(week)]['RvNR'] = both

        data_exog = sm.add_constant(data_exog)
        if 'scfa' in key:
            x = x.drop('Caproate', axis = 1)
        if 'toxin' in key:
            x = x.iloc[:,-4:]

        # metab = x.columns.values[0]
        res_dict = {}
        for metab in x.columns.values:
            res_dict[metab] = {}
            model = sm.OLS(x[metab], data_exog)
            results = model.fit()
            if week != 0:
                cont_res = results.t_test([0,1,0])
            else:
                cont_res = results.t_test([0,1])
            df = cont_res.summary_frame()
            res_dict[metab]['p, outcome'] = df['P>|t|']['c0']
        #     res_dict[metab]['t_out'] = df['t']['c0']
            res_dict[metab]['coef, outcome'] = results.params['Outcome\n(R=1,NR=0)']
            if week != 0:
                df_2 = results.t_test([0,0,1]).summary_frame()
                res_dict[metab]['p, treatment'] = df_2['P>|t|']['c0']
                res_dict[metab]['coef, treatment'] = results.params['Drug\n(Vanc=1, Metro=0)']
            
        final_df = pd.DataFrame(res_dict).T
        reject, corrected, a1, a2 = multipletests(final_df['p, outcome'], alpha=.05, method='fdr_bh')
        final_df.insert(1, 'FDR, Outcome', corrected)
        
        if week != 0:
            reject2, corrected2, a12, a22 = multipletests(final_df['p, treatment'], alpha=.05, method='fdr_bh')
            final_df.insert(4, 'FDR, Treatment', corrected2)
            
        if not os.path.isdir(path_to_save+'univariate_analysis_control/' + key):
            os.mkdir(path_to_save+'univariate_analysis_control/' + key)
        if plot:
            for metab in df2.index.values:
                if df2['BH corrected'][metab] < .05:
                    try:
                        plot_metab_over_time(metab, df2['BH corrected'][metab], dl.keys[key], path_to_save + 'univariate_analysis/' + 
                                             key + '/' + str(week) + '_')
                        print('plotted ' + metab)
                    except:
                        plot_metab_over_time(metab, df2['BH corrected'][metab], dl.keys['metabs'], path_to_save + 'univariate_analysis/' + 
                                             key + '/' + str(week) + '_')
                        print('plotted ' + metab)                    
        
#         df.to_csv(path_to_save+'univariate_analysis/' + key + '/ranksum_' + key + str(week) + '.csv')
        final_df.to_csv(path_to_save+'univariate_analysis_control/' + key + '/OLS_' + key + str(week) + '.csv')
#         plt.figure()
#         plt.hist(df2['BH corrected'], bins = np.arange(0,1.05,.05))
#         plt.xlabel('p-values')
#         plt.ylabel('Frequency')
#         plt.title(key + ', week ' + str(week))
#         plt.savefig(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week) + '.pdf')
#         plt.close()
        
        
for key in data_to_save.keys():
    for key2 in data_to_save[key].keys():
        for key3 in data_to_save[key][key2].keys():
            val = pd.DataFrame(data_to_save[key][key2][key3])
            if os.path.isfile(path_to_save_data + '/' + 'univ_analysis_wcontrol.xlsx'):
                with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_wcontrol.xlsx', mode='a', engine='openpyxl') as writer:
                    val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
            else:
                with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_wcontrol.xlsx', mode = 'w', engine='openpyxl') as writer:
                    val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))

In [211]:
taxa_labels = pd.read_csv(path_to_save + 'scripts/inputs/' + 'taxa_labels.csv', index_col = [0])


In [226]:
taxa_labels = pd.read_csv(path_to_save + 'scripts/inputs/' + 'taxa_labels.csv', index_col = [0])

key = '16s'
new_path = path_to_save+'univariate_analysis_control/' + key + '/'
old_path = path_to_save+'univariate_analysis/' + key + '/'
for file in os.listdir(path_to_save+'univariate_analysis_control/' + key):
    if file == '.DS_Store':
        continue
    print(file)
    df_new = pd.read_csv(new_path + file, index_col = [0])
    df_old = pd.read_csv(old_path + file, index_col = [0])
    
    new_sig = df_new.loc[df_new['padj']<0.05]
    
    old_sig = df_old.loc[df_old['padj']<0.05]
    
    print('# new sig: ' + str(len(new_sig)))
    print('# old sig: ' + str(len(old_sig)))
    both = set(new_sig.index.values).intersection(set(old_sig.index.values))
    print('# common: ' + str(len(both)))
    print('')
    print('Became significant: ')
    for metab in new_sig.index.values:
        if metab not in old_sig.index.values:
            print(taxa_labels.loc[metab]['labels'])
            print(taxa_labels.loc[metab]['taxa_rdp'])
            print(str(new_sig['Direction'][metab]))
        
    print('')
    print('No longer significant: ')
    for metab in old_sig.index.values:
        if metab not in new_sig.index.values:
            print(taxa_labels.loc[metab]['labels'])
            print(taxa_labels.loc[metab]['taxa_rdp'])
            print(str(old_sig['Direction'][metab]))
            
    print('')
    print('')

deseq2_16sNon-recurrer1_2.csv
# new sig: 30
# old sig: 12
# common: 8

Became significant: 
ASV_19
Bacteria; Bacteroidetes; Bacteroidia; Bacteroidales; Prevotellaceae; Prevotella; histicola
Higher in week 1
ASV_133
Bacteria; Firmicutes; Erysipelotrichia; Erysipelotrichales; Erysipelotrichaceae; NA; NA
Higher in week 2
ASV_179
Bacteria; Firmicutes; Bacilli; Lactobacillales; Enterococcaceae; NA; NA
Higher in week 1
ASV_202
Bacteria; Firmicutes; Clostridia; Clostridiales; Lachnospiraceae; NA; NA
Higher in week 2
ASV_84
Bacteria; Firmicutes; Clostridia; Clostridiales; Lachnospiraceae; NA; NA
Higher in week 2
ASV_204
Bacteria; Firmicutes; Clostridia; Clostridiales; Lachnospiraceae; NA; NA
Higher in week 2
ASV_93
Bacteria; Firmicutes; Clostridia; Clostridiales; Lachnospiraceae; Clostridium_XlVa; NA
Higher in week 2
ASV_211
Bacteria; Firmicutes; Clostridia; Clostridiales; Lachnospiraceae; Ruminococcus2; NA
Higher in week 2
ASV_64
Bacteria; Firmicutes; Clostridia; Clostridiales; Lachnospiracea

IsADirectoryError: [Errno 21] Is a directory: '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/univariate_analysis_control/16s/deseq2'

In [227]:
key = 'metabs'
new_path = path_to_save+'univariate_analysis_control/' + key + '/'
old_path = path_to_save+'univariate_analysis/' + key + '/'
for file in os.listdir(path_to_save+'univariate_analysis_control/' + key):
    if file == '.DS_Store':
        continue
    print(file)
    df_new = pd.read_csv(new_path + file, index_col = [0])
    df_old = pd.read_csv(old_path + 'ttest' + file.split('OLS')[1], index_col = [0])
    
    try:
        new_sig = df_new.loc[df_new['FDR, Outcome']<0.05]
    except:
        new_sig = df_new.loc[df_new['FDR, Week']<0.05]
    
    old_sig = df_old.loc[df_old['BH corrected']<0.05]
    
    print('# new sig: ' + str(len(new_sig)))
    print('# old sig: ' + str(len(old_sig)))
    both = set(new_sig.index.values).intersection(set(old_sig.index.values))
    print('# common: ' + str(len(both)))
    print('')
    print('Became significant: ')
    for metab in new_sig.index.values:
        if metab not in old_sig.index.values:
            try:
                print(metab + '; ' + str(new_sig.loc[metab]['coef, week']))
            except:
                print(metab + '; ' + str(new_sig.loc[metab]['coef, outcome']))
        
    print('')
    print('No longer significant: ')
    for metab in old_sig.index.values:
        if metab not in new_sig.index.values:
            print(metab + '; ' + str(old_sig.loc[metab]['test statistic']))
            
    print('')
    print('')

OLS_metabs_Recurrer_0v1.csv
# new sig: 41
# old sig: 43
# common: 35

Became significant: 
1-dihomo-linolenylglycerol (20:3); -1.298225890228779
3-hydroxyoleate*; -1.1840291355608006
carboxy-methyl-arginine; 1.35363924049508
ergosterol; 1.243982529536968
octadecanedioate (C18); -1.2839923164723226
pheophorbide A; 1.4288320549429334

No longer significant: 
hexadecenedioate (C16:1-DC)*; -3.7012276213516433
5-methylcytosine; 3.6249394250980473
undecenoylcarnitine (C11:1); 3.621789496241214
3-phenylpropionate (hydrocinnamate); -3.621132383896155
epicatechin; 3.5892745324519244
1,1-kestotetraose; 3.5223227899752145
1-methylhistamine; 3.483949891503816
octadecadienedioate (C18:2-DC)*; -3.403243609885444


OLS_metabs_Recurrer_1v2.csv
# new sig: 0
# old sig: 0
# common: 0

Became significant: 

No longer significant: 


OLS_metabs_Non-recurrer_0v1.csv
# new sig: 229
# old sig: 228
# common: 222

Became significant: 
1-dihomo-linolenylglycerol (20:3); -0.7014740918052359
4-methylhexanoylglutam

In [6]:
# # Univariate analysis for metabolites and SCFAs
# path_to_save_data=save_path + '/GenFigure&AnalysesData/'

# weeks = [0,1,2]
# week_pairs = [(0,1),(1,2)]
# plot = False
# data_to_save = {}
# for key in ['metabs', 'scfa']:
#     data_to_save[key] = {}
#     if not os.path.isdir(path_to_save + 'univariate_analysis/' + key):
#         os.mkdir(path_to_save + 'univariate_analysis/' + key)
#     for outcome in ['Recurrer', 'Non-recurrer']:
#         if outcome not in data_to_save[key].keys():
#             data_to_save[key][outcome] = {}
#         for week_pair in week_pairs:
#             x0, y0 = dl.week[key][week_pair[0]]['x'], dl.week[key][week_pair[0]]['y']
#             x0 = x0.loc[y0.values == outcome]
#             x1, y1 = dl.week[key][week_pair[1]]['x'], dl.week[key][week_pair[1]]['y']
#             x1 = x1.loc[y1.values == outcome]

#             xboth = list(set(x0.columns.values).intersection(set(x1.columns.values)))
#             x = pd.concat([x0, x1])
#             if 'scfa' in key:
#                 x = x.drop('Caproate', axis = 1)
#             y = [week_pair[0]]*x0.shape[0] + [week_pair[1]]*x1.shape[0]
#             y = pd.Series((np.array(y)==week_pair[1]).astype('float'), index = x.index.values)
            
#             yy = pd.DataFrame(y, columns = ['Outcome'])
#             both = pd.concat([x, yy], 1)
#             data_to_save[key][outcome]['Week ' + str(week_pair[0]) + 'v' + str(week_pair[1])] = both
            
#             df = univariate_w_chi2(x,y)
#             df2 = univariate_w_chi2(x,y, method = 'ttest')
#             if not os.path.isdir(path_to_save+'univariate_analysis/' + key ):
#                 os.mkdir(path_to_save+'univariate_analysis/' + key)

#             df.insert(2,'Direction', np.zeros(df.shape[0]).astype(str))
#             df2.insert(2,'Direction', np.zeros(df2.shape[0]).astype(str))

#             # Higher in cleared group (0) or first week
#             df["Direction"].loc[df["direction"].astype('float64')<0] = 'Higher in week ' + str(week_pair[0])
#             df2["Direction"].loc[df2["direction"].astype('float64')<0] = 'Higher in week ' + str(week_pair[0])
#             # Higher in recurred group (1) or second week
#             df["Direction"].loc[df["direction"].astype('float64')>0] = 'Higher in week ' + str(week_pair[1])
#             df2["Direction"].loc[df2["direction"].astype('float64')>0] = 'Higher in week ' + str(week_pair[1])

#             df.to_csv(path_to_save+'univariate_analysis/' + key + '/ranksum_' + key + str(week_pair[0]) + '_' + 
#                        str(week_pair[1]) + '_' + outcome + '.csv')
#             df2.to_csv(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week_pair[0]) + '_' + 
#                        str(week_pair[1]) + '_' + outcome + '.csv')
#             plt.figure()
#             plt.hist(df2['BH corrected'], bins = np.arange(0,1.05,.05))
#             plt.xlabel('p-values')
#             plt.ylabel('Frequency')
#             plt.title(key + ', week ' + str(week_pair))
#             plt.savefig(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week_pair[0]) + '_' + 
#                        str(week_pair[1]) + '_' + outcome + '.pdf')
#             plt.close()
# #             with pd.ExcelWriter(path_save + 'Table 5 - Predictive Results.xlsx', mode = 'a') as writer:
# #                 df.to_excel(writer, sheet_name = 'A. Results')
        
#     for week in weeks:
#         if week not in data_to_save[key].keys():
#             data_to_save[key]['Week ' + str(week)] = {}

#         if not isinstance(week, list):
#             x, y = dl.week[key][week]['x'], dl.week[key][week]['y']
            
#             yy = pd.Series((np.array(y)=='Recurrer').astype('float'), index = x.index.values)
#             yy = pd.DataFrame(yy, columns = ['Outcome'])
#             both = pd.concat([x, yy], 1)
#             data_to_save[key]['Week ' + str(week)]['RvNR'] = both
#             if 'scfa' in key:
#                 x = x.drop('Caproate', axis = 1)
#             if 'toxin' in key:
#                 x = x.iloc[:,-4:]
#             df = univariate_w_chi2(x,y)
#             df2 = univariate_w_chi2(x,y, method = 'ttest')
#         else:
#             x,y,t = get_slope_data(dl.week[key], week)
#             if 'scfa' in key:
#                 x = x.drop('Caproate', axis = 1)
#             if 'toxin' in key:
#                 x = x.iloc[:,-4:]
#             df = univariate_w_chi2(x,y)
#             df2 = univariate_w_chi2(x,y,method = 'ttest')
#         y = pd.DataFrame(y, columns = ['Outcome'])
#         both = pd.concat([x, yy], 1)
#         data_to_save[key]['Week ' + str(week)]['RvNR'] = both
            
#         if not os.path.isdir(path_to_save+'univariate_analysis/' + key):
#             os.mkdir(path_to_save+'univariate_analysis/' + key)
#         if plot:
#             for metab in df2.index.values:
#                 if df2['BH corrected'][metab] < .05:
#                     try:
#                         plot_metab_over_time(metab, df2['BH corrected'][metab], dl.keys[key], path_to_save + 'univariate_analysis/' + 
#                                              key + '/' + str(week) + '_')
#                         print('plotted ' + metab)
#                     except:
#                         plot_metab_over_time(metab, df2['BH corrected'][metab], dl.keys['metabs'], path_to_save + 'univariate_analysis/' + 
#                                              key + '/' + str(week) + '_')
#                         print('plotted ' + metab)                    
        
#         df.insert(2,'Direction', np.zeros(df.shape[0]).astype(str))
#         df2.insert(2,'Direction', np.zeros(df2.shape[0]).astype(str))
        
#         # Higher in cleared group (0) or first week
#         df["Direction"].loc[df["direction"].astype('float64')<0] = 'Decreased Risk'
#         df2["Direction"].loc[df2["direction"].astype('float64')<0] = 'Decreased Risk'

#         # Higher in recurred group (1) or second week
#         df["Direction"].loc[df["direction"].astype('float64')>0] = 'Increased Risk'
#         df2["Direction"].loc[df2["direction"].astype('float64')>0] = 'Increased Risk'
        
#         df.to_csv(path_to_save+'univariate_analysis/' + key + '/ranksum_' + key + str(week) + '.csv')
#         df2.to_csv(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week) + '.csv')
#         plt.figure()
#         plt.hist(df2['BH corrected'], bins = np.arange(0,1.05,.05))
#         plt.xlabel('p-values')
#         plt.ylabel('Frequency')
#         plt.title(key + ', week ' + str(week))
#         plt.savefig(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week) + '.pdf')
#         plt.close()
        
        
# for key in data_to_save.keys():
#     for key2 in data_to_save[key].keys():
#         for key3 in data_to_save[key][key2].keys():
#             val = pd.DataFrame(data_to_save[key][key2][key3])
#             if os.path.isfile(path_to_save_data + '/' + 'univ_analysis_data.xlsx'):
#                 with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_data.xlsx', mode='a', engine='openpyxl') as writer:
#                     val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
#             else:
#                 with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_data.xlsx', mode = 'w', engine='openpyxl') as writer:
#                     val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))

# Univariate analysis for metabolites & SCFAs, drug as response variable

In [9]:
# # Univariate analysis for metabolites and SCFAs
# path_to_save_data=save_path + '/GenFigure&AnalysesData/'

# weeks = [0,1,2]
# week_pairs = [(0,1),(1,2)]
# plot = False
# data_to_save = {}
# for key in ['metabs', 'scfa']:
#     data_to_save[key] = {}
#     if not os.path.isdir(path_to_save + 'univariate_analysis_drug/' + key):
#         os.mkdir(path_to_save + 'univariate_analysis_drug/' + key)
#     for outcome in ['vanc', 'flagyl']:
#         if outcome not in data_to_save[key].keys():
#             data_to_save[key][outcome] = {}
#         for week_pair in week_pairs:
#             x0 = dl.week[key][week_pair[0]]['x']
#             y0 = df_demo['Drug'].copy()
#             y0.index = [str(ix) + '-' + str(week_pair[0]) for ix in y0.index.values]
#             y0 = y0.loc[x0.index.values]
#             x0 = x0.loc[y0.values == outcome]
            
#             x1 = dl.week[key][week_pair[1]]['x']
#             y1 = df_demo['Drug'].copy()
#             y1.index = [str(ix) + '-' + str(week_pair[1]) for ix in y1.index.values]
#             y1 = y1.loc[x1.index.values]
#             x1 = x1.loc[y1.values == outcome]
            
#             xboth = list(set(x0.columns.values).intersection(set(x1.columns.values)))
#             x = pd.concat([x0, x1])
#             if 'scfa' in key:
#                 x = x.drop('Caproate', axis = 1)
#             y = [week_pair[0]]*x0.shape[0] + [week_pair[1]]*x1.shape[0]
#             y = pd.Series((np.array(y)==week_pair[1]).astype('float'), index = x.index.values)
            
#             yy = pd.DataFrame(y, columns = ['Outcome'])
#             both = pd.concat([x, yy], 1)
#             data_to_save[key][outcome]['Week ' + str(week_pair[0]) + 'v' + str(week_pair[1])] = both
            
#             df = univariate_w_chi2(x,y)
#             df2 = univariate_w_chi2(x,y, method = 'ttest')
#             if not os.path.isdir(path_to_save+'univariate_analysis_drug/' + key ):
#                 os.mkdir(path_to_save+'univariate_analysis_drug/' + key)

#             df.insert(2,'Direction', np.zeros(df.shape[0]).astype(str))
#             df2.insert(2,'Direction', np.zeros(df2.shape[0]).astype(str))

#             # Higher in cleared group (0) or first week
#             df["Direction"].loc[df["direction"].astype('float64')<0] = 'Higher in week ' + str(week_pair[0])
#             df2["Direction"].loc[df2["direction"].astype('float64')<0] = 'Higher in week ' + str(week_pair[0])
#             # Higher in recurred group (1) or second week
#             df["Direction"].loc[df["direction"].astype('float64')>0] = 'Higher in week ' + str(week_pair[1])
#             df2["Direction"].loc[df2["direction"].astype('float64')>0] = 'Higher in week ' + str(week_pair[1])

#             df.to_csv(path_to_save+'univariate_analysis_drug/' + key + '/ranksum_' + key + str(week_pair[0]) + '_' + 
#                        str(week_pair[1]) + '_' + outcome + '.csv')
#             df2.to_csv(path_to_save+'univariate_analysis_drug/' + key + '/ttest_' + key + str(week_pair[0]) + '_' + 
#                        str(week_pair[1]) + '_' + outcome + '.csv')
#             plt.figure()
#             plt.hist(df2['BH corrected'], bins = np.arange(0,1.05,.05))
#             plt.xlabel('p-values')
#             plt.ylabel('Frequency')
#             plt.title(key + ', week ' + str(week_pair))
#             plt.savefig(path_to_save+'univariate_analysis_drug/' + key + '/ttest_' + key + str(week_pair[0]) + '_' + 
#                        str(week_pair[1]) + '_' + outcome + '.pdf')
#             plt.close()
# #             with pd.ExcelWriter(path_save + 'Table 5 - Predictive Results.xlsx', mode = 'a') as writer:
# #                 df.to_excel(writer, sheet_name = 'A. Results')
        
#     for week in weeks:
#         if week not in data_to_save[key].keys():
#             data_to_save[key]['Week ' + str(week)] = {}

# #         if not isinstance(week, list):
#         x = dl.week[key][week]['x']
#         y = df_demo['Drug'].copy()
#         y.index = [str(ix) + '-' + str(week) for ix in y.index.values]
#         y = y.loc[x.index.values]

#         y = pd.Series((np.array(y)=='vanc').astype('float'), index = x.index.values)
#         yy = pd.DataFrame(yy, columns = ['Outcome'])
#         both = pd.concat([x, yy], 1)
#         data_to_save[key]['Week ' + str(week)]['VancVsMet'] = both
#         if 'scfa' in key:
#             x = x.drop('Caproate', axis = 1)
#         if 'toxin' in key:
#             x = x.iloc[:,-4:]
#         df = univariate_w_chi2(x,y)
#         df2 = univariate_w_chi2(x,y, method = 'ttest')
# #         else:
# #             x,y,t = get_slope_data(dl.week[key], week)
# #             if 'scfa' in key:
# #                 x = x.drop('Caproate', axis = 1)
# #             if 'toxin' in key:
# #                 x = x.iloc[:,-4:]
# #             df = univariate_w_chi2(x,y)
# #             df2 = univariate_w_chi2(x,y,method = 'ttest')
#         y = pd.DataFrame(y, columns = ['Outcome'])
#         both = pd.concat([x, yy], 1)
#         data_to_save[key]['Week ' + str(week)]['VancVsMet'] = both
            
#         if not os.path.isdir(path_to_save+'univariate_analysis_drug/' + key):
#             os.mkdir(path_to_save+'univariate_analysis_drug/' + key)
#         if plot:
#             for metab in df2.index.values:
#                 if df2['BH corrected'][metab] < .05:
#                     try:
#                         plot_metab_over_time(metab, df2['BH corrected'][metab], dl.keys[key], path_to_save + 'univariate_analysis_drug/' + 
#                                              key + '/' + str(week) + '_')
#                         print('plotted ' + metab)
#                     except:
#                         plot_metab_over_time(metab, df2['BH corrected'][metab], dl.keys['metabs'], path_to_save + 'univariate_analysis_drug/' + 
#                                              key + '/' + str(week) + '_')
#                         print('plotted ' + metab)                    
        
#         df.insert(2,'Direction', np.zeros(df.shape[0]).astype(str))
#         df2.insert(2,'Direction', np.zeros(df2.shape[0]).astype(str))
        
#         # Higher in cleared group (0) or first week
#         df["Direction"].loc[df["direction"].astype('float64')<0] = 'Decreased Risk'
#         df2["Direction"].loc[df2["direction"].astype('float64')<0] = 'Decreased Risk'

#         # Higher in recurred group (1) or second week
#         df["Direction"].loc[df["direction"].astype('float64')>0] = 'Increased Risk'
#         df2["Direction"].loc[df2["direction"].astype('float64')>0] = 'Increased Risk'
        
#         df.to_csv(path_to_save+'univariate_analysis_drug/' + key + '/ranksum_' + key + str(week) + '.csv')
#         df2.to_csv(path_to_save+'univariate_analysis_drug/' + key + '/ttest_' + key + str(week) + '.csv')
#         plt.figure()
#         plt.hist(df2['BH corrected'], bins = np.arange(0,1.05,.05))
#         plt.xlabel('p-values')
#         plt.ylabel('Frequency')
#         plt.title(key + ', week ' + str(week))
#         plt.savefig(path_to_save+'univariate_analysis_drug/' + key + '/ttest_' + key + str(week) + '.pdf')
#         plt.close()
        
        
# for key in data_to_save.keys():
#     for key2 in data_to_save[key].keys():
#         for key3 in data_to_save[key][key2].keys():
#             val = pd.DataFrame(data_to_save[key][key2][key3])
#             if os.path.isfile(path_to_save_data + '/' + 'univ_analysis_drug_response.xlsx'):
#                 with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_drug_response.xlsx', mode='a', engine='openpyxl') as writer:
#                     val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
#             else:
#                 with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_drug_response.xlsx', mode = 'w', engine='openpyxl') as writer:
#                     val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))

## Save compact univariate analysis to supplemental tables

In [206]:
from string import ascii_uppercase
path_univariate = path_to_save + 'univariate_analysis_control/'
path_save = save_path + '/Supplemental Tables/'
taxa_labels = pd.read_csv(path_to_save + 'scripts/inputs/' + 'taxa_labels.csv', index_col = [0])

k = 3
for dtype in os.listdir(path_univariate):
    if dtype == 'toxin':
        continue
    labs = []
    df_out = {}

    fname = 'Univariate Analysis Cntrl for Drug.xlsx'

    i = 0
    if dtype == '.DS_Store':
        continue
        
    if '.csv' in dtype:
        continue
    for file in os.listdir(path_univariate + dtype):
        if 'kruskal' in file or 'ranksum' in file or '[' in file or '.DS_Store' in file:
            continue
        if '.csv' not in file:
            continue
#         print(i)
#         print(file)
        timepoint = file.split('.')[0].split(dtype)[1]
        if 'rer' in timepoint:
            if dtype == '16s':
                label = timepoint.split('rer')[0] + 'rer; Week ' + 'vs'.join(timepoint.split('rer')[1].split('_'))

            else:
                label = timepoint.split('_')[-1] + '; Week ' + 'vs'.join(timepoint.split('_')[:-1])
            lab_out = ('Intra-group',label)
        else:
            label = 'Week ' + timepoint + '\n Recurrer vs Non-recurrer'
            lab_out = ('Inter-group',label)
        
        df = pd.read_csv(path_univariate + dtype + '/' + file, index_col = [0])
        labs.append(lab_out)
        if dtype == '16s':
            df = df.sort_values(by = 'padj')
#             if np.sum(df['padj']<0.1)==0:
#                 df = df.iloc[:10,:]
#             else:
            df = df.loc[df['padj']<0.1]
            seq_labels = taxa_labels['labels'].loc[df.index.values]
            for ix in df.index.values:
                df_out[(lab_out[0], lab_out[1], ix)] = {'Labels': seq_labels.loc[ix], 'FDR': df['padj'].loc[ix], 
                                                        'Direction': df['Direction'].loc[ix], 
                          'log2fold': df['log2fold'].loc[ix],
                         'Taxonomy-RDP': taxa_labels['taxa_rdp'].loc[ix], 
                          }
#             df_out[lab_out] = pd.DataFrame(df_out[lab_out])
#             df_out[lab_out] = df_out[lab_out].set_index('Labels')
        elif dtype == 'metabs':
            if 'v' in timepoint:
                df = df.sort_values(by = 'FDR, Week')
                df = df.loc[df['FDR, Week']<0.1]
                for ix in df.index.values:
                    df_out[(lab_out[0], lab_out[1], ix)] = {'FDR': df['FDR, Week'].loc[ix], 
                              'Coefficient': df['coef, week'].loc[ix], 
                             'Super Pathway': dl.col_mat_mets['SUPER_PATHWAY'].loc[ix],
                             'Sub Pathway': dl.col_mat_mets['SUB_PATHWAY'].loc[ix]}
            else:
                df = df.sort_values(by = 'FDR, Outcome')
                df = df.loc[df['FDR, Outcome']<0.1]
                for ix in df.index.values:
                    df_out[(lab_out[0], lab_out[1], ix)] = {'FDR': df['FDR, Outcome'].loc[ix], 
                              'Coefficient': df['coef, outcome'].loc[ix], 
                             'Super Pathway': dl.col_mat_mets['SUPER_PATHWAY'].loc[ix],
                             'Sub Pathway': dl.col_mat_mets['SUB_PATHWAY'].loc[ix]}
        else:
            if 'v' in timepoint:
                df = df.sort_values(by = 'FDR, Week')
                df = df.loc[df['FDR, Week']<0.1]
                for ix in df.index.values:
                    df_out[(lab_out[0], lab_out[1], ix)] = {'FDR': df['FDR, Week'].loc[ix], 
                              'Coefficient': df['coef, week'].loc[ix]}
            else:
                df = df.sort_values(by = 'FDR, Outcome')
                df = df.loc[df['FDR, Outcome']<0.1]
                for ix in df.index.values:
                    df_out[(lab_out[0], lab_out[1], ix)] = {'FDR': df['FDR, Outcome'].loc[ix], 
                              'Coefficient': df['coef, outcome'].loc[ix]}
        
    df_out = pd.DataFrame(df_out).T.sort_index()
    if os.path.exists(path_save + fname):
        with pd.ExcelWriter(path_save + fname, mode = 'a') as writer:
            df_out.to_excel(writer, sheet_name = dtype.capitalize())
    else:
        with pd.ExcelWriter(path_save + fname) as writer:
            df_out.to_excel(writer, sheet_name = dtype.capitalize())
#         i+= 1
    k += 1

## Save full univariate analysis to Data folder

In [208]:
from string import ascii_uppercase
path_univariate = path_to_save + 'univariate_analysis_control/'
path_save = save_path + '/Data/Univariate Analysis - Treatment Control/'
if not os.path.isdir(path_save):
    os.mkdir(path_save)
taxa_labels = pd.read_csv(path_to_save + 'scripts/inputs/' + 'taxa_labels.csv', index_col = [0])

k = 3
for dtype in os.listdir(path_univariate):
    fname = 'Table ' + str(k + 1) + '-' + dtype.capitalize() + ' Univariate Analysis.xlsx'

    
    if dtype == '.DS_Store':
        continue
    if '.csv' in dtype:
        os.rename(path_univariate + dtype, path_save + dtype)
        continue
    i = 0
    for file in os.listdir(path_univariate + dtype):
        if 'kruskal' in file or 'ranksum' in file or '[' in file or '.DS_Store' in file:
            continue
        if '.csv' not in file:
            continue
#         print(i)
#         print(file)
        
        df = pd.read_csv(path_univariate + dtype + '/' + file, index_col = [0])
        if dtype == '16s':
            seq_labels = taxa_labels['labels'].loc[df.index.values]
            df_out = {'FDR': df['padj'], 'Direction': df['Direction'], 
                      'log2fold': df['log2fold'],'Taxonomy-Silva': taxa_labels['taxa_silva'].loc[df.index.values], 
                     'Taxonomy-RDP': taxa_labels['taxa_rdp'].loc[df.index.values], 
                      'Sequences': df.index.values, 'Labels': seq_labels}
            df_out = pd.DataFrame(df_out)
            df_out = df_out.set_index('Labels')
        elif dtype == 'metabs':
            if '1v2' in file or '0v1' in file:
                df_out = {'FDR': df['FDR, Week'], 'Coefficient': df['coef, week'], 
                          'PubChem#': dl.col_mat_mets['PUBCHEM'].loc[df.index.values],
                         'KEGG#': dl.col_mat_mets['PUBCHEM'].loc[df.index.values], 'HMDB#': 
                          dl.col_mat_mets['HMDB'].loc[df.index.values], 
                         'Super Pathway': dl.col_mat_mets['SUPER_PATHWAY'].loc[df.index.values],
                         'Sub Pathway': dl.col_mat_mets['SUB_PATHWAY'].loc[df.index.values], 'names':df.index.values}
            else:
                df_out = {'FDR': df['FDR, Outcome'], 'Coefficient': df['coef, outcome'], 
                          'PubChem#': dl.col_mat_mets['PUBCHEM'].loc[df.index.values],
                         'KEGG#': dl.col_mat_mets['PUBCHEM'].loc[df.index.values], 'HMDB#': 
                          dl.col_mat_mets['HMDB'].loc[df.index.values], 
                         'Super Pathway': dl.col_mat_mets['SUPER_PATHWAY'].loc[df.index.values],
                         'Sub Pathway': dl.col_mat_mets['SUB_PATHWAY'].loc[df.index.values], 'names':df.index.values}
            df_out = pd.DataFrame(df_out)
            df_out = df_out.set_index('names')
        else:
            if '1v2' in file or '0v1' in file:
                df_out = {'FDR': df['FDR, Week'], 
                          'Coefficient': df['coef, week']}
            else:
                df_out = {'FDR': df['FDR, Outcome'], 
                          'Coefficient': df['coef, outcome']}
            df_out = pd.DataFrame(df_out)
        
        timepoint = file.split('.')[0].split(dtype)[1]

        if 'rer' in timepoint:
            if dtype == '16s':
                label = timepoint.split('rer')[0] + 'rer;Week' + 'vs'.join(timepoint.split('rer')[1].split('_'))
            else:
                label = timepoint.split('_')[-1] + ';Week' + 'vs'.join(timepoint.split('_')[:-1])
        else:
            label = 'Week' + timepoint + ';R_vs_NR'
        
        nm = ascii_uppercase[i]
        
        if os.path.exists(path_save + fname):
            with pd.ExcelWriter(path_save + fname, mode = 'a') as writer:
                df_out.to_excel(writer, sheet_name = nm + '.' + label)
        else:
            with pd.ExcelWriter(path_save + fname) as writer:
                df_out.to_excel(writer, sheet_name = nm + '.' + label)
        i+= 1
    k += 1

In [142]:
file

'OLS_scfa1.csv'